In [16]:
import scipy.sparse as sps
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender
from Utils.Evaluator import EvaluatorHoldout
import Utils.Reader as Reader
from Data_manager.split_functions.split_train_validation_random_holdout import\
    split_train_in_two_percentage_global_sample
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Utils.Evaluator import EvaluatorHoldout
import Utils.Reader as Reader

#URM_all=Reader.load_URM(values_to_replace={0.01:0.02})
URM_all=Reader.read_train_csr(values_to_replace={0:0.03})
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all ,train_percentage=0.80)


from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GeneralizedMergedHybridRecommender import GeneralizedMergedHybridRecommender
import numpy as np
import os
import itertools
best_result=0
best={}
from tqdm.notebook import tqdm
import numpy
parent_dir =  "../"
write_path = os.path.join(parent_dir, "Testing_Results/grid.csv")

Unique UserID in the URM are 41629
Unique UserID in the URM and ICM are 41629
5826506


In [ ]:
alphaL=numpy.arange(0.3, 0.5, 0.01)
betaL=numpy.arange(0.00, 0.2, 0.01)
gammaL=numpy.arange(0.00, 0.2, 0.05)
blockfact=numpy.arange(0.01, 0.05, 0.01)
rp3=RP3betaRecommender(URM_train)
rp3.fit()

ials=IALSRecommender(URM_train)
ials.fit()

itemKNN=ItemKNNCFRecommender(URM_train)
itemKNN.fit()
slim =SLIM_BPR_Cython(URM_train)
slim.fit()
p3=P3alphaRecommender(URM_train)
p3.fit()
with open(write_path, "w+") as d:
    for group_id in tqdm(range(17, 23)):
        for x in itertools.product(alphaL,betaL):
            alpha = x[0]
            beta = x[1]
            #gamma = x[2]
            blockfactor = 0.04
            profile_length = np.ediff1d(sps.csr_matrix(URM_all).indptr)
            sorted_users = np.argsort(profile_length)
            block_size = int(len(profile_length) * blockfactor)
            start_pos = group_id * block_size
            end_pos = min((group_id + 1) * block_size, len(profile_length))

            users_in_group = sorted_users[start_pos:end_pos]

            users_in_group_p_len = profile_length[users_in_group]
            users_not_in_group_flag = np.isin(sorted_users, users_in_group, invert=True)
            users_not_in_group = sorted_users[users_not_in_group_flag]

            w1 = len(users_in_group) / len(profile_length)
            w2 = len(users_not_in_group) / len(profile_length)
            from Utils.Evaluator import EvaluatorHoldout
            from Recommenders.GeneralizedMergedHybridRecommender import GeneralizedMergedHybridRecommender
            import numpy as np

            '''
            higher_recommender = GeneralizedMergedHybridRecommender(
                URM_train=URM_train,
                recommenders=[
                    itemKNN

                ],
                verbose=False
            )
            higher_recommender.fit(
                alphas=[
                    alpha,beta,
                    1 - alpha - beta
                ]
            )

            '''
            lower_recommender = GeneralizedMergedHybridRecommender(
                URM_train=URM_train,
                recommenders=[
                    ials,p3,
                     rp3

                ],
                verbose=False
            )
            lower_recommender.fit(
                alphas=[
                    alpha,beta,
                    1 - alpha-beta
                ]
            )
            cutoff = 10
            evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[cutoff], ignore_users=users_not_in_group)
            result_df, _ = evaluator_test.evaluateRecommender(itemKNN)

            evaluator_test2 = EvaluatorHoldout(URM_test, cutoff_list=[cutoff], ignore_users=users_in_group)
            result_df2, _ = evaluator_test2.evaluateRecommender(lower_recommender)
            r2 = result_df2["MAP"][10]
            r = result_df["MAP"][10]

            result = r * w1 + r2 * w2
            if result > best_result:
                print("New Result")
                best_result = result
                best = {
                    "alpha:": str(alpha),
                    "beta": str(beta),
                    "groupid:": str(group_id),
                    "block_factor": str(blockfactor),
                    "MAP": str(best_result),
                    "res1:": str(r),
                    "res2:": str(r2)
                }
                d.write(str(best))
                d.write('\n')

In [40]:
best

{'alpha:': '0.4',
 'beta': '0.1',
 'gamma:': '0.01',
 'groupid:': '21',
 'block_factor': '0.04',
 'MAP': '0.02865889284113962',
 'res1:': '0.05653882453882443',
 'res2:': '0.02749734529142875'}

In [42]:
result

0.028391045514270147